In [22]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [30]:
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
n_embd     = 384
n_layer    = 6
n_head = 6
n_layer = 6
dropout    = 0.2
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
with open('input.txt', 'r') as fh:
    text = fh.read()

In [11]:
vocab = sorted(list(set(text)))
vocab_size = len(vocab)
stoi = {x : i for i, x in enumerate(vocab)}
itos = {i : x for i, x in enumerate(vocab)}
encode = lambda s : [stoi[c] for c in s]
decode = lambda l : ''.join([itos[i] for i in l])

assert "abc" == decode(encode("abc"))

In [12]:
data = torch.tensor(encode(text))
n = int(0.9*len(data))
train_data = data[:n]
val_data   = data[n:]

In [14]:
def batch_get(split):
    data = train_data if split == "train" else val_data
    ix   = torch.randint(len(data) - block_size, (batch_size,))
    x    = torch.stack([data[i : i + block_size] for i in ix])
    y    = x.copy()
    x    = x.to(device)
    y    = y.to(device)
    return x, y

In [47]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key   = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):               # x : (B, T, C)
        k = self.key(x)                 # k : (B, T, head_size)
        q = self.query(x)               # q : (B, T, head_size)
        v = self.value(x)               # v : (B, T, head_size)

        wei = q @ v.transpose(-2,-1) * (k.shape[-1]**-0.5) # (B, T, head_size) * (B, head_size, T) = (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)

        # perform weighted aggregation of values
        out = wei @ v # (B, T, T) @ (B, T, head_size) => (B, T, head_size)
        return out

In [35]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj  = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], axis=-1)
        out = self.dropout(out)
        return out

In [44]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [32]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa   = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1  = nn.LayerNorm(n_embd)
        self.ln2  = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [33]:
class GPTLanguageModel(nn.Module):
    def __init__(self, ):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.pos_embedding_table   = nn.Embedding(block_size, n_embd)
        self.blocks                = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f                  = nn.LayerNorm(n_embd)
        self.lm_head               = nn.Linear(n_embd, vocab_size)
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.pos_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, target)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond) # B, T
            probs = F.softmax(logits, dim=-1)
            # sample next character based on predictions
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat([idx, idx_next], dim=-1)
        return idx

In [45]:
model = GPTLanguageModel()
m = model.to(device)

In [46]:
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

10.788929 M parameters
